In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statistics as es 
import csv
import re 
import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Categorical  features transformed by regression from ENEM (Student Grain) and Teacher Census (Teache grain) to School Grain. Here are only schools in our scope

In [2]:
#Reading transformed features from Docente Table in Censo and Student in Enem. Code in Grain_transformation file
trans_regress_att = pd.read_csv("trans_regress_att.csv")
trans_regress_att.rename(columns={'TP_COR_RACA':'QT_COR_RACA', 'TITULACAO': 'QT_TITULACAO', 'NU_IDADE':'QT_IDADE',
                                 'TP_SEXO':'QT_SEXO'}, inplace = True)

In [3]:
trans_regress_att.columns

Index(['QT_TITULACAO', 'CO_ESCOLA', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006',
       'Q007', 'QT_IDADE', 'QT_SEXO', 'QT_COR_RACA', 'Q005', 'Q008', 'Q009',
       'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018',
       'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'TARGET'],
      dtype='object')

### School Census Data 2018

In [4]:
#Reading all columns. In The last time, in other file(grain_transformation) 
#we read only necessaries features to filter works.
censo18a=pd.read_csv("~/data/censo2018/ESCOLAS.CSV", sep='|', encoding="iso-8859-2")

In [5]:
censo18 = censo18a.copy()

In [6]:
#censo18.drop(['IN_SALA_DIRETORIA', 'IN_EQUIP_TV', 'IN_EQUIP_DVD',
#       'IN_EQUIP_IMPRESSORA', 'IN_COMPUTADOR'], axis=1, inplace=True)

### Merging Student  and Teacher transformed grain + School data

In [7]:
#merging it with censo school data
dataset = pd.merge(trans_regress_att, censo18, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='left')

In [8]:
# It already has a lot of features... let's take out those are irrelevant to domain and group those remaining 
infrastructure = (["IN_SALA_DIRETORIA","IN_PREDIO_COMPARTILHADO","IN_AGUA_FILTRADA",
                    "IN_AGUA_REDE_PUBLICA","IN_AGUA_POCO_ARTESIANO","IN_AGUA_CACIMBA",
                    "IN_AGUA_FONTE_RIO","IN_ENERGIA_REDE_PUBLICA","IN_ENERGIA_GERADOR",
                    "IN_ENERGIA_OUTROS","IN_ESGOTO_REDE_PUBLICA","IN_ESGOTO_FOSSA",
                    "IN_LIXO_COLETA_PERIODICA","IN_LIXO_QUEIMA","IN_LIXO_JOGA_OUTRA_AREA",
                    "IN_LIXO_RECICLA","IN_LIXO_ENTERRA","IN_LIXO_OUTROS",
                    "IN_SALA_PROFESSOR","IN_LABORATORIO_INFORMATICA","IN_LABORATORIO_CIENCIAS",
                    "IN_SALA_ATENDIMENTO_ESPECIAL","IN_QUADRA_ESPORTES_COBERTA",
                    "IN_QUADRA_ESPORTES_DESCOBERTA","IN_QUADRA_ESPORTES","IN_COZINHA","IN_BIBLIOTECA",
                    "IN_SALA_LEITURA","IN_BIBLIOTECA_SALA_LEITURA","IN_BANHEIRO_FORA_PREDIO",
                    "IN_BANHEIRO_DENTRO_PREDIO","IN_BANHEIRO_EI","IN_BANHEIRO_PNE","IN_DEPENDENCIAS_PNE",
                    "IN_SECRETARIA","IN_BANHEIRO_CHUVEIRO","IN_REFEITORIO","IN_DESPENSA","IN_ALMOXARIFADO",
                    "IN_AUDITORIO","IN_PATIO_COBERTO","IN_PATIO_DESCOBERTO","IN_ALOJAM_ALUNO",
                    "IN_ALOJAM_PROFESSOR","IN_AREA_VERDE","IN_LAVANDERIA","IN_DEPENDENCIAS_OUTRAS",
                    "QT_SALAS_EXISTENTES","QT_SALAS_UTILIZADAS","QT_EQUIP_TV","QT_EQUIP_VIDEOCASSETE",
                    "QT_EQUIP_DVD","QT_EQUIP_PARABOLICA","QT_EQUIP_COPIADORA","QT_EQUIP_RETROPROJETOR",
                    "QT_EQUIP_IMPRESSORA","QT_EQUIP_IMPRESSORA_MULT","QT_EQUIP_SOM","QT_EQUIP_MULTIMIDIA",
                    "QT_EQUIP_FAX","QT_EQUIP_FOTO","QT_COMPUTADOR","QT_COMP_ADMINISTRATIVO","QT_COMP_ALUNO",
                    "IN_BANDA_LARGA",'IN_INTERNET',"QT_FUNCIONARIOS","IN_ALIMENTACAO", "TP_DEPENDENCIA", 'TP_LOCALIZACAO' ])


hypothesis = (['Q001', 'Q002', 'QT_TITULACAO'])



socioeconomic =(["Q003","Q004","Q005","Q006","Q007","Q008","Q009","Q010","Q011","Q012","Q013","Q014",
                "Q015","Q016","Q017","Q018","Q019","Q020","Q021","Q022","Q023","Q024","Q025",'QT_COR_RACA', "QT_IDADE" ])



target = (['TARGET'])

In [9]:
dataset = dataset[hypothesis + socioeconomic + infrastructure+target]

In [10]:
dataset.head(2)


,Q001,Q002,QT_TITULACAO,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,QT_COR_RACA,QT_IDADE,IN_SALA_DIRETORIA,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_LIXO_COLETA_PERIODICA,IN_LIXO_QUEIMA,IN_LIXO_JOGA_OUTRA_AREA,IN_LIXO_RECICLA,IN_LIXO_ENTERRA,IN_LIXO_OUTROS,IN_SALA_PROFESSOR,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_COZINHA,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_BANHEIRO_EI,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,IN_SECRETARIA,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_DESPENSA,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_ALUNO,IN_ALOJAM_PROFESSOR,IN_AREA_VERDE,IN_LAVANDERIA,IN_DEPENDENCIAS_OUTRAS,QT_SALAS_EXISTENTES,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_VIDEOCASSETE,QT_EQUIP_DVD,QT_EQUIP_PARABOLICA,QT_EQUIP_COPIADORA,QT_EQUIP_RETROPROJETOR,QT_EQUIP_IMPRESSORA,QT_EQUIP_IMPRESSORA_MULT,QT_EQUIP_SOM,QT_EQUIP_MULTIMIDIA,QT_EQUIP_FAX,QT_EQUIP_FOTO,QT_COMPUTADOR,QT_COMP_ADMINISTRATIVO,QT_COMP_ALUNO,IN_BANDA_LARGA,IN_INTERNET,QT_FUNCIONARIOS,IN_ALIMENTACAO,TP_DEPENDENCIA,TP_LOCALIZACAO,TARGET
0,0.304298,0.176767,0.224855,0.169913,0.377012,6.592593,0.120046,0.212366,1.029630,2.000000,0.000000,0.000000,1.000000,0.029630,0.659259,0.000000,0.007407,0.0,1.000000,1.000000,0.059259,1.00000,1.933333,0.992593,0.014815,0.970370,3.000000,18.348148,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,18.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,17.0,7.0,10.0,1.0,1.0,122.0,1.0,2,1,0
1,0.021968,0.036028,0.187727,0.071404,0.067398,4.433735,0.076538,0.212366,1.192771,1.987952,0.108434,0.457831,1.036145,0.048193,0.216867,0.012048,0.156627,0.0,0.975904,1.361446,0.602410,0.73494,2.253012,1.000000,0.204819,0.626506,2.963855,17.843373,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,16.0,6.0,10.0,0.0,1.0,43.0,1.0,2,1,0


In [11]:
#Missing Values
k=pd.DataFrame()
k['NULL'] = dataset.isnull().sum()
k[k.NULL >0].T

,IN_PREDIO_COMPARTILHADO,IN_BANDA_LARGA
NULL,50,409


In [12]:
#Every shools that have not Fast Inernet have not single internet too?
a = dataset.loc[dataset['IN_BANDA_LARGA'].isnull()]
a.IN_INTERNET.value_counts()

0.0    409
Name: IN_INTERNET, dtype: int64

In [13]:
#by context analysis will fill with these two features
dataset.fillna(0, axis=1, inplace=True)

In [14]:
#Split categories features to better handle
category = dataset.filter(regex = 'IN_|TP_', axis=1).columns
def category_type(df):
    for i in category:
          df[i] = df[i].astype('category')


category_type(dataset)

In [15]:
#Drop out categorical feature that the mode is upper the thresold
def mode_high_frequencie(df):
    ammount =0
    total_cat = df.select_dtypes(include=['category'])
    print("Number of categorical features: ", total_cat.shape[1])
    for i in total_cat:
        mode = df[i].mode()[0]
        threshold = 0.9
        freq = df[(df[i]== mode)].shape[0]
        value = freq/df.shape[0]
        if value > threshold:
            ammount +=1
            print("drop out", [i], "mode = ", mode )
            df.drop([i], inplace = True, axis=1)
    print("Total Dropped: ",ammount)
    print ("Remainning: ", total_cat.shape[1]-ammount)


mode_high_frequencie(dataset)

Number of categorical features:  52
drop out ['IN_SALA_DIRETORIA'] mode =  1.0
drop out ['IN_PREDIO_COMPARTILHADO'] mode =  0.0
drop out ['IN_AGUA_REDE_PUBLICA'] mode =  1.0
drop out ['IN_AGUA_POCO_ARTESIANO'] mode =  0.0
drop out ['IN_AGUA_CACIMBA'] mode =  0.0
drop out ['IN_AGUA_FONTE_RIO'] mode =  0.0
drop out ['IN_ENERGIA_REDE_PUBLICA'] mode =  1.0
drop out ['IN_ENERGIA_GERADOR'] mode =  0.0
drop out ['IN_ENERGIA_OUTROS'] mode =  0.0
drop out ['IN_LIXO_COLETA_PERIODICA'] mode =  1.0
drop out ['IN_LIXO_QUEIMA'] mode =  0.0
drop out ['IN_LIXO_JOGA_OUTRA_AREA'] mode =  0.0
drop out ['IN_LIXO_RECICLA'] mode =  0.0
drop out ['IN_LIXO_ENTERRA'] mode =  0.0
drop out ['IN_LIXO_OUTROS'] mode =  0.0
drop out ['IN_SALA_PROFESSOR'] mode =  1.0
drop out ['IN_COZINHA'] mode =  1.0
drop out ['IN_BIBLIOTECA_SALA_LEITURA'] mode =  1.0
drop out ['IN_BANHEIRO_DENTRO_PREDIO'] mode =  1.0
drop out ['IN_BANHEIRO_EI'] mode =  0.0
drop out ['IN_SECRETARIA'] mode =  1.0
drop out ['IN_ALOJAM_ALUNO'] mode = 

In [16]:
dataset.columns

Index(['Q001', 'Q002', 'QT_TITULACAO', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007',
       'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
       'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025',
       'QT_COR_RACA', 'QT_IDADE', 'IN_AGUA_FILTRADA', 'IN_ESGOTO_REDE_PUBLICA',
       'IN_ESGOTO_FOSSA', 'IN_LABORATORIO_INFORMATICA',
       'IN_LABORATORIO_CIENCIAS', 'IN_SALA_ATENDIMENTO_ESPECIAL',
       'IN_QUADRA_ESPORTES_COBERTA', 'IN_QUADRA_ESPORTES_DESCOBERTA',
       'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_SALA_LEITURA',
       'IN_BANHEIRO_FORA_PREDIO', 'IN_BANHEIRO_PNE', 'IN_DEPENDENCIAS_PNE',
       'IN_BANHEIRO_CHUVEIRO', 'IN_REFEITORIO', 'IN_DESPENSA',
       'IN_ALMOXARIFADO', 'IN_AUDITORIO', 'IN_PATIO_COBERTO',
       'IN_PATIO_DESCOBERTO', 'IN_AREA_VERDE', 'QT_SALAS_EXISTENTES',
       'QT_SALAS_UTILIZADAS', 'QT_EQUIP_TV', 'QT_EQUIP_VIDEOCASSETE',
       'QT_EQUIP_DVD', 'QT_EQUIP_PARABOLICA', 'QT_EQUIP_COPIADORA',
       'QT_EQUIP_

In [17]:
#writing dataset in disk to AED Analysis. in AED file
dataset.to_csv('dataset.csv')

###  After AED and Context Analysis in "AED Notebook" file

In [18]:
#drop hight correlation find out in AED Analisys.
dataset.drop(['QT_SALAS_EXISTENTES','QT_COMP_ADMINISTRATIVO','QT_COMPUTADOR'], inplace = True, axis=1)

In [19]:
#drop Invariance features in AED Analysis
dataset.drop(['QT_EQUIP_RETROPROJETOR','QT_EQUIP_VIDEOCASSETE', 'QT_EQUIP_PARABOLICA', 'QT_EQUIP_FAX'], inplace = True, axis=1)

In [20]:
#update list infrastructure list
infrastructure = dataset.loc[:, dataset.columns.isin(infrastructure)].columns.to_list()
quantitative_infra = dataset[infrastructure].filter(regex = 'QT_', axis=1).columns.to_list()

In [21]:
dataset[quantitative_infra+hypothesis+socioeconomic].dtypes

QT_SALAS_UTILIZADAS         float64
QT_EQUIP_TV                 float64
QT_EQUIP_DVD                float64
QT_EQUIP_COPIADORA          float64
QT_EQUIP_IMPRESSORA         float64
QT_EQUIP_IMPRESSORA_MULT    float64
QT_EQUIP_SOM                float64
QT_EQUIP_MULTIMIDIA         float64
QT_EQUIP_FOTO               float64
QT_COMP_ALUNO               float64
QT_FUNCIONARIOS             float64
Q001                        float64
Q002                        float64
QT_TITULACAO                float64
Q003                        float64
Q004                        float64
Q005                        float64
Q006                        float64
Q007                        float64
Q008                        float64
Q009                        float64
Q010                        float64
Q011                        float64
Q012                        float64
Q013                        float64
Q014                        float64
Q015                        float64
Q016                        

In [22]:
#outliers numerical fatures, based in EAD - Q0...

dataset[quantitative_infra+hypothesis+socioeconomic] = dataset[quantitative_infra+hypothesis+socioeconomic].apply(lambda x:
                                                                                                                  x.clip(upper = (np.percentile(x, 97.5))))
dataset[(['Q005', 'Q008','Q009','Q012', 'Q018', 
          'Q019', 'Q020', 'Q021', 'Q022', 'QT_COR_RACA','QT_IDADE'])] = dataset[(['Q005', 'Q008','Q009','Q012', 'Q018', 
                                                                'Q019', 'Q020', 'Q021', 'Q022', 'QT_COR_RACA','QT_IDADE'])].apply(lambda x:
                                                                                                                x.clip(lower = (np.percentile(x, 2.5))))

In [23]:
dataset.head(3)


,Q001,Q002,QT_TITULACAO,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,QT_COR_RACA,QT_IDADE,IN_AGUA_FILTRADA,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_DESPENSA,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_AREA_VERDE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_EQUIP_IMPRESSORA_MULT,QT_EQUIP_SOM,QT_EQUIP_MULTIMIDIA,QT_EQUIP_FOTO,QT_COMP_ALUNO,IN_BANDA_LARGA,QT_FUNCIONARIOS,TARGET
0,0.304298,0.176767,0.224855,0.169913,0.377012,5.200000,0.120046,0.212366,1.029630,2.000000,0.000000,0.000000,1.000000,0.029630,0.659259,0.000000,0.007407,0.0,1.000000,1.000000,0.31250,1.000000,1.933333,0.992593,0.014815,0.905650,2.900000,18.348148,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,18.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,10.0,1.0,122.0,0
1,0.021968,0.036028,0.187727,0.071404,0.067398,4.433735,0.076538,0.212366,1.192771,1.987952,0.108434,0.457831,1.036145,0.048193,0.216867,0.012048,0.156627,0.0,0.975904,1.361446,0.60241,0.734940,2.253012,1.000000,0.204819,0.626506,2.900000,17.843373,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,10.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,10.0,0.0,43.0,0
2,0.068503,0.056787,0.371262,0.094391,0.045330,4.262295,0.056863,0.223329,1.032787,2.163934,0.147541,0.262295,1.000000,0.098361,0.049180,0.000000,0.163934,0.0,1.000000,0.918033,0.57377,0.983607,1.819672,0.983607,0.147541,0.426230,2.491803,18.311475,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0,1.0,11.0,0.0,79.0,0


In [24]:
dataset.to_csv('dataset_final.csv', index = False)

In [25]:
print(len(infrastructure),
len(hypothesis),
len(socioeconomic))

34 3 25
